In [3]:
!pip install d2l
#6.4.1. 多输入通道
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历 “X” 和 “K” 的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

     |████████████████████████████████| 81kB 5.5MB/s 


In [4]:
#我们可以构造与 图6.4.1 中的值相对应的输入张量 X 和核张量 K，以验证互相关运算的输出。
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [5]:
#6.4.2. 多输出通道
#如下所示，我们实现一个计算多个通道的输出的互相关函数。
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [6]:
#通过将核张量 K 与 K+1 （ K 中每个元素加  1  ）和 K+2 连接起来，
#构造了一个具有 3 个输出通道的卷积核。
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [7]:
#下面，我们对输入张量 X 与卷积核张量 K 执行互相关运算。现在的输出包含3个通道，
#第一个通道的结果与先前输入张量 X 和多输入单输出通道的结果一致。
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [8]:
#6.4.3. 1×1卷积层
#下面，我们使用全连接层实现  1×1  卷积。 请注意，我们需要对输入和输出的数据形状进行微调。
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)  # 全连接层中的矩阵乘法
    return Y.reshape((c_o, h, w))

In [9]:
#当执行  1×1  卷积运算时，上述函数相当于先前实现的互相关函数corr2d_multi_in_out。让我们用一些样本数据来验证这一点。
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

assert float(torch.abs(Y1 - Y2).sum()) < 1e-6